In [1]:
import open3d as o3d
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import copy as cp
#import open3d.core as o3c
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
import time
import functions
from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten
from mpl_toolkits.mplot3d import Axes3D

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#load pcd file, filter, downsample
#pcdn = o3d.io.read_point_cloud("loop.pcd")

voxel_size = 0.4
pcd = o3d.io.read_point_cloud("final_cropped_ground_align2.pcd")
pcd.estimate_normals()
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=0.8)

pcd = pcd.select_by_index(ind)
pcd = pcd.voxel_down_sample(voxel_size=0.1)
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(40)
#o3d.visualization.draw_geometries([pcd])

In [3]:
pcd_flat = functions.filter_by_normal(pcd)

o3d.visualization.draw_geometries([pcd_flat])

In [4]:
oboxes = pcd_flat.detect_planar_patches(
normal_variance_threshold_deg=50,
coplanarity_deg=85,
outlier_ratio=0.75,
min_plane_edge_length=2,
min_num_points=10,
search_param=o3d.geometry.KDTreeSearchParamKNN(knn=50))

print("Detected {} patches".format(len(oboxes)))

geometries = []
meshes = []
for obox in oboxes:

    mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox, scale=[1, 1, 0.0001])
    mesh.paint_uniform_color(obox.color)
    mesh.compute_triangle_normals()
    mesh.compute_vertex_normals()
    normals = np.asarray(mesh.vertex_normals)

    
    meshes.append(mesh)
    geometries.append(obox)



Detected 23 patches


In [5]:
hor_patches, ver_patches = functions.devide_meshes_hor_ver(meshes)
o3d.visualization.draw_geometries(ver_patches)
o3d.visualization.draw_geometries(hor_patches)
print(len(hor_patches), len(ver_patches))

7 16


In [6]:


corr, corr_tuples = functions.group_corresponding_meshes([],[],hor_patches, "horizontal")

corr, corr_tuples = functions.group_corresponding_meshes(corr, corr_tuples, ver_patches, "vertical")


In [7]:




midpoints, marker_meshes = functions.find_midpoints(corr_tuples)

print(midpoints)
o3d.visualization.draw_geometries(corr + marker_meshes)

[array([-7.05462401e+00,  5.37344191e+00,  5.75408022e-03]), array([-1.29458319e+01, -7.82807673e-03,  5.27914373e-03]), array([-1.27030232e+01,  1.14223320e-02,  4.89931328e-03]), array([-3.74719922e+01,  5.80080373e-01,  4.33622586e-03]), array([-2.29517470e+01,  2.78227621e-01,  5.01004857e-03]), array([-1.84816841, -0.19480336,  0.00522409]), array([-1.47052726e+01,  8.89535841e-02,  4.98860158e-03]), array([-3.87831660e+01,  3.93533610e-01,  5.28642415e-03]), array([-1.03315074e+01,  3.29897735e-01,  5.94279572e-03]), array([-1.10702387e+01, -9.63713830e-04,  5.15520578e-03])]


In [8]:
bb = pcd_flat.get_axis_aligned_bounding_box()
#bb.color = np.asarray([0,0,1])
#uniform_pc = create_uniform_pc_from_bb(bb, 0.2, [1,0,0])
hull, _ = pcd_flat.compute_convex_hull()

uniform_pc = functions.hull_to_uniform_pc(hull, 0.2, [1,0,0])

o3d.visualization.draw_geometries([pcd_flat, uniform_pc, bb])

pcd_flat = functions.create_uniform_pc(pcd_flat, 0.1, [0,0,0])
o3d.visualization.draw_geometries([pcd_flat])

In [9]:
# search multiple iterations until boundary found
search_radius = np.ceil(np.sqrt((voxel_size*2)**2+(voxel_size*2)**2)*10000)/10000

uniform_pc.paint_uniform_color([1,0,0])
pcd_flat.paint_uniform_color([0,0,0])

PointCloud with 3985 points.

In [10]:


void_area, known_points = functions.extract_void_area(uniform_pc, pcd_flat, midpoints)
o3d.visualization.draw_geometries([void_area, pcd_flat]+marker_meshes)

In [11]:
valid_area = uniform_pc.select_by_index(list(known_points))
valid_area_voxel = o3d.geometry.VoxelGrid.create_from_point_cloud(valid_area, 0.2)
o3d.visualization.draw_geometries([valid_area_voxel, pcd_flat])

In [12]:
o3d.io.write_point_cloud("valid_area.pcd", valid_area)
o3d.io.write_point_cloud("pcd_flat.pcd", pcd_flat)

True